# Everymind - FireData
Neste notebook, apresentamos a descrição do processo de desenvolver um modelo preditivo de classificação capaz de identificar quais funcionários têm tendências a sair ou permanecer na empresa.

# Integrantes

*   Alexandre Fonseca de Souza
*   Bruno Otavio Bezerra de Meira
*   Filipi Enzo Siqueira Kikuchi
*   Gabriela de Morais da Silva
*   João Pedro Sartori Garcia de Alcaraz
*   Lucas Conti Pereira
*   Sofia Moreiras Pimazzoni


Primeiramente, utilizaremos o arquivo que contém a base de dados dos colaboradores

#1.Preparação do algoritmo
Para que o algoritmo funcione da forma desejada, primeiro instalamos e importamos funções e bibliotecas que auxiliam na execução de algumas tarefas. Todas as importações estão listadas abaixo:

In [ ]:
# Instalando o moment
!pip install moment

# Importação de bibliotecas
# Pandas
import pandas as pd

# scikitlearn
from sklearn.preprocessing import LabelEncoder

# Datetime
from datetime import datetime
from datetime import date

# Moment
import moment

#1.1 Importação dos dados
Para que o algoritmo consiga performar, precisamos importar o arquivo contendo os dados, a fim de tratá-los e posteriormente fazer analises mais acuradas. Como possuímos três tabelas com dados diferentes, separamo-nos em 3 dataframes diferentes.

In [ ]:
# Importando o arquivo que contém a base de dados

# Dataframe da tabela "Everymind"
df_everymind = pd.read_excel('/content/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', sheet_name = "Everymind")

# Dataframe da tabela "Reconhecimento"
df_reconhecimento = pd.read_excel('/content/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', sheet_name = "Reconhecimento")

# Dataframe da tabela "Reconhecimento"
df_ambiente = pd.read_excel('/content/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', sheet_name = "Ambiente de Trabalho 27.07")

#2. Formatação
 Nesta seção, vamos tratar os dados a fim de padronizá-los para que sejam aceitos e melhor utilizados pelo algoritmo a partir de funções que modificam a forma do dado. No momento, estamos trabalhando com alguns tipos de dados, sendo eles:



*   Dados relacionadas a tempo
*   Dados relaciondados a nome



##2.1 Variáveis relacionadas a tempo
As manipulações para dados deste tipo envolvem a conversão do formato para o padrão "YYYY-MM-DD" (Ex: 2020-09-14) e a inclusão de valores em cédulas vazias para efeitos de cálculo (Na coluna "Dt Saida", quando não consta a data de saída, atribuimos o dia atual para podermos calcular o número de dias na empresa). Sofrem essa alteração os dados presentes nas colunas "Dt Admissao", "Dt Nascimento" e "Dt Saida". As transformações seguem abaixo:



In [ ]:
#------------------------------------Tabela Everymind--------------------------

# Formata a data de admissao no formato ano/mes/dia
for i in range(0, len(df_everymind['Dt Admissao'])):

  df_everymind['Dt Admissao'][i] = str(df_everymind['Dt Admissao'][i]).replace('00:00:00', '')
  df_everymind['Dt Admissao'][i] = str(df_everymind['Dt Admissao'][i]).replace('/', '-')
  df_everymind['Dt Admissao'][i] = moment.date(df_everymind['Dt Admissao'][i]).format("YYYY-MM-DD")

df_everymind['Dt Admissao'] = pd.to_datetime(df_everymind['Dt Admissao'], format="%Y/%m/%d")

# Formata a data de nascimento no formato ano/mes/dia

for i in range(0, len(df_everymind['Dt Nascimento'])):

  df_everymind['Dt Nascimento'][i] = moment.date(df_everymind['Dt Nascimento'][i]).format("YYYY-MM-DD")

df_everymind['Dt Nascimento'] = pd.to_datetime(df_everymind['Dt Nascimento'], format="%Y/%m/%d")


# Formata a data de saída, sendo que se o valor for vazio, atribui a data atual como data de saída

df_everymind['Dt Saida'] = pd.to_datetime(df_everymind['Dt Saida'], format="%Y/%m/%d")

df_saida = df_everymind['Dt Saida'].isnull()

# Verifica se a cédula é vazia e se for, adiciona a data de hoje

for i in range(0,len(df_saida)):
  if df_saida[i] == True:
    df_everymind['Dt Saida'][i] = pd.Timestamp.today()

df_everymind['Dt Saida'] = pd.to_datetime(df_everymind['Dt Saida'], format="%Y/%m/%d")

#------------------------------------Tabela Reconhecimento--------------------------

# Coloca a coluna "Data de Admissao" no formato ano/mes/dia

for i in range(0, len(df_reconhecimento['Data de Admissão'])):

  df_reconhecimento['Data de Admissão'][i] = str(df_reconhecimento['Data de Admissão'][i]).replace('00:00:00', '')
  df_reconhecimento['Data de Admissão'][i] = str(df_reconhecimento['Data de Admissão'][i]).replace('/', '-')
  df_reconhecimento['Data de Admissão'][i] = moment.date(df_reconhecimento['Data de Admissão'][i]).format("YYYY-MM-DD")

df_reconhecimento['Data de Admissão'] = pd.to_datetime(df_reconhecimento['Data de Admissão'], format="%Y/%m/%d")

# Formata a data de Vigência no formato ano/mes/dia
for i in range(0, len(df_reconhecimento['Data Vigência'])):
  
  df_reconhecimento['Data Vigência'][i] = str(df_reconhecimento['Data Vigência'][i]).replace('00:00:00', '')
  df_reconhecimento['Data Vigência'][i] = str(df_reconhecimento['Data Vigência'][i]).replace('/', '-')
  df_reconhecimento['Data Vigência'][i] = moment.date(df_reconhecimento['Data Vigência'][i]).format("YYYY-MM-DD")

df_reconhecimento['Data Vigência'] = pd.to_datetime(df_reconhecimento['Data Vigência'], format="%Y/%m/%d")

display(df_everymind)

##2.2 Variáveis relacionadas a nomes

Para que o programa funcione corretamente, todos os dados que contêm nomes e/ou informações baseadas em texto foram padronizados removendo todos os espaços em branco (Exemplo: "Funcional Pl" vira "FuncionalPl").



In [ ]:
# ------------------------------------Tabela Everymind--------------------------


# Pega todas as colunas do dataframe com type "object" e tira todos os espaços em branco
dtg = df_everymind.select_dtypes(include = "object").columns

for j in dtg:
  df_everymind[j] = df_everymind[j].str.replace(' ', '')

# Pega todos os campos vazios da coluna de "Tipo Saida" e coloca "ColaboradorAtivo" nela.

dfsaii = df_everymind['Tipo Saida'].isnull()

for i in range(0,len(dfsaii)):
  if dfsaii[i] == True:
    df_everymind['Tipo Saida'][i] = "ColaboradorAtivo"

df_everymind['Tipo Saida'] = dfsaii

# Cria uma coluna nova
df_everymind['PessoaColaboradora'] = ""

#Pega a coluna "Nome completo" tira o texto, deixa apenas o número do colaborador e adiciona na coluna criada "PessoaColaboradora" 

for i in range(0, len(df_everymind['Nome Completo'])):
  df_everymind['PessoaColaboradora'][i] = str(df_everymind['Nome Completo'][i]).replace('PessoaColaboradora', '')


# ------------------------------------Tabela Reconhecimento--------------------------


# Pega todas as colunas com type "object" e tira todos os espaços para ficar padronizado
df_reconhecimento_types = df_reconhecimento.select_dtypes(include = "object").columns

for j in df_reconhecimento_types:
  df_reconhecimento[j] = df_reconhecimento[j].str.replace(' ', '')

# Cria uma nova coluna "Pessoa colaboradora"
df_reconhecimento['PessoaColaboradora'] = ""

# Transforma a coluna "Codinome" em number, tirando os textos e colocando o numero do colaborador na coluna "PessoaColaboradora"
for i in range(0, len(df_reconhecimento['Codinome'])):
  df_reconhecimento['PessoaColaboradora'][i] = str(df_reconhecimento['Codinome'][i]).replace('PessoaColaboradora', '')


# ------------------------------------Tabela Ambiente de Trabalho 27.07--------------------------


# Pega todas as colunas com type "object" e tira todos os espaços para ficar padronizado
df_ambiente_types = df_ambiente.select_dtypes(include = "object").columns

for j in df_ambiente_types:
  df_ambiente[j] = df_ambiente[j].str.replace(' ', '')

# Pega todos os campos vazios da coluna com a Keyword e atribui 0
keywords = ['Pulou', 'Muito Insatisfeito', 'Insatisfeito', 'Neutro', 'Satisfeito', 'Muito Satisfeito']

for i in keywords:
  
  df_ambiente_satisfaction = df_ambiente[i].isnull()

  for j in range(0,len(df_ambiente_satisfaction)):
    if df_ambiente_satisfaction[j] == True:
      df_ambiente[i][j] = "0"

display(df_ambiente)

#3. Derivação de novos atributos

A partir da limpeza dos dados e do relacionamento entre eles, conseguimos gerar novos elementos que facilitam as analises e auxiliam na tomada de decisões da IA. A partir dos dados obtidos, obtivemos os seguintes atributos:


*   Número de dias na empresa
*   Idade do colaborador
*   Dias para reconhecimento entre data de admissão e vigência




In [ ]:
# Derivação das datas em novos atributos

# Função que calcula os dias entre duas datas
def numOfDays(date1, date2):
    return (date2-date1).days

# Cria uma coluna de idade e uma de numero de dias na empresa na empresa 
df_everymind['Idade'] = ""
df_everymind['NumeroDias'] = ""

# Faz o calculo da idade e armazena 
for i in range(0, len(df_everymind['Dt Nascimento'])):
  if df_everymind['Tipo Saida'][i] == str('ColaboradorAtivo'):
    if date.today().month>=(df_everymind['Dt Nascimento'][i]).month:
      df_everymind['Idade'][i] = date.today().year - (df_everymind['Dt Nascimento'][i]).year
    else:
      df_everymind['Idade'][i] = (date.today().year - (df_everymind['Dt Nascimento'][i]).year)-1
  else:
    if df_everymind['Dt Saida'][i].month>=(df_everymind['Dt Nascimento'][i]).month:
      df_everymind['Idade'][i] = date.today().year - (df_everymind['Dt Nascimento'][i]).year
    else:
      df_everymind['Idade'][i] = (df_everymind['Dt Saida'][i].year - df_everymind['Dt Nascimento'][i].year)-1

# Faz o calculo do numero de dias na empresa de cada colaborador

date1 = df_everymind['Dt Admissao']
date2 = df_everymind['Dt Saida']

for i in range(0, len(date2)):
  df_everymind['NumeroDias'][i] = str(numOfDays(date1[i], date2[i]))

#Cria uma coluna "DiasParaReconhecimento"
df_reconhecimento['DiasParaReconhecimento'] = ""

#Calcula os dias que o empregador entrou na empresa até receber reconhecimento
date_reconhecimento1 = df_reconhecimento['Data de Admissão']
date_reconhecimento2 = df_reconhecimento['Data Vigência']
for i in range(0, len(date_reconhecimento2)):
  df_reconhecimento['DiasParaReconhecimento'][i] = str(numOfDays(date_reconhecimento1[i], date_reconhecimento2[i]))

#4. Encoders

As variáveis do tipo _string_ são reconhecidas com dificuldade pelo modelo, por conta disso, é necessário transformá-las em números (tipo _int_ e _float_). Existem duas maneiras de se fazer o "encoding", sendo elas:


* Label Encoding
* One Hot Encoding

##4.1 Label Encoding

O Label Encoding transforma as features em números ordenados, ou seja, em ordem. É normalmente usada quando uma feature é mais importante que outra. No projeto a única que identificamos que possui ordem é a escolaridade (na tabela Everymind).

In [ ]:
# Cria um novo Data Frame para a coluna de Escolaridade e raliza o processo de Label Encoding
data_frame_esco = pd.DataFrame(df, columns=["Escolaridade "]) 
df_esco_atual = data_frame_esco.replace(['EnsinoMédioIncompleto',
                                         'EnsinoMédio', 
                                         'Técnico', 
                                         'Superiorincompleto', 
                                         'Graduação', 
                                         'PósGraduação', 
                                         'Mestrado'], 
                                         [0,1,2,3,4,5,6])

##4.2 One Hot Encoding

O One Hot Encoding cria uma coluna para cada valor e faz uma atribuição do valor 1 (um) para a coluna correspondente da amostra e 0 (zero) para as outras. Dessa forma, uma linha tem apenas um valor. Essa forma de _encoding_ costuma ser mais utilizada, pois não atribui uma ordem às features. Nós usamos o One Hot Encoding em todas as tabelas.

In [ ]:
# ------------------------------------Tabela Everymind--------------------------

# A função "get_dummies()" realiza o One Hot Encoding da coluna desejada. 
one_hot_cargos = pd.get_dummies(df_everymind["Cargo"])
one_hot_saida = pd.get_dummies(df_everymind["Tipo Saida"])
one_hot_genero = pd.get_dummies(df_everymind["Genero"])
one_hot_etnia = pd.get_dummies(df_everymind["Etnia"])
one_hot_civil = pd.get_dummies(df_everymind["Estado Civil"]) 
one_hot_estado = pd.get_dummies(df_everymind["Estado"])
one_hot_cidade = pd.get_dummies(df_everymind["Cidade"])
one_hot_area = pd.get_dummies(df_everymind["Area"])
one_hot_colaborador = pd.get_dummies(df_everymind["PessoaColaboradora"])


# ------------------------------------Tabela Reconhecimento--------------------------

# A mesma função acima se aplica nessa tabela
data_frame_codinome = pd.get_dummies(df_reconhecimento["Codinome"])
data_frame_situacao = pd.get_dummies(df_reconhecimento["Situação"])
data_frame_novocargo = pd.get_dummies(df_reconhecimento["Novo Cargo"])
data_frame_motivo = pd.get_dummies(df_reconhecimento["Motivo"])
data_frame_funcao = pd.get_dummies(df_reconhecimento["Alterou Função"])

#5. Concatenar os dados

Para facilitar o desenvolvimento do projeto, foi decidido criar tabelas novas com as modificações feitas.

In [ ]:
# bru, não sei o que essas linhas significam, mas vou colocar aqui pq sei que elas são importantes
collaborator = df['PessoaColaboradora']
registration = df['Matrícula']
age = df['Idade']
companyTime = df['NumeroDias']
monthlySalary = df['Salario Mês']

In [ ]:
# ------------------------------------Tabela Everymind--------------------------

#Cria uma tabela nova com os dados formatados (rodar quando os dados tiverem finalizados)
newData = pd.DataFrame()
newData = pd.concat([newData,registration], axis = 1)
newData = pd.concat([newData,collaborator], axis = 1)
newData = pd.concat([newData,age], axis = 1)
newData = pd.concat([newData,companyTime], axis = 1)
newData = pd.concat([newData,one_hot_saida], axis = 1)
newData = pd.concat([newData,one_hot_cargos], axis = 1)
newData = pd.concat([newData,monthlySalary], axis = 1)
newData = pd.concat([newData,one_hot_genero], axis = 1)
newData = pd.concat([newData,one_hot_civil], axis = 1)
newData = pd.concat([newData,df_esco_atual], axis = 1)
newData = pd.concat([newData,one_hot_estado], axis = 1)
newData = pd.concat([newData,one_hot_cidade], axis = 1)
newData = pd.concat([newData,one_hot_area], axis = 1)


newData.to_excel('dados_formatados_aba_everymind.xlsx')


# ------------------------------------Tabela Reconhecimento--------------------------

#Cria uma tabela nova com os dados formatados (rodar quando os dados tiverem finalizados)
newData_reconhecimento = pd.DataFrame()
newData_reconhecimento = pd.concat([newData_one,data_frame_codinome], axis = 1)
newData_reconhecimento = pd.concat([newData_one,data_frame_situacao], axis = 1)
newData_reconhecimento = pd.concat([newData_one,data_frame_novocargo], axis = 1)
newData_reconhecimento = pd.concat([newData_one,data_frame_motivo], axis = 1)
newData_reconhecimento = pd.concat([newData_one,data_frame_funcao], axis = 1)

display(newData_one)